In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Fermilab FAST example

In [2]:
from impact import Impact, template_dir
import os

In [3]:
ifile = os.path.join(template_dir, 'fermilab_fast/ImpactT.in')
os.path.exists(ifile)

True

In [8]:
int('0.0')

ValueError: invalid literal for int() with base 10: '0.0'

In [10]:
# Make Impact object
I = Impact(ifile, verbose=True)

ValueError: invalid literal for int() with base 10: '0.0'

In [ ]:
# Show a MAD-style printout of each element in the lattice
I.print_lattice()

In [ ]:
# Look at an element by name
I.ele['emfield_cylindrical_1']

In [ ]:
# Change its phase
I.ele['emfield_cylindrical_1']['theta0_deg'] = 120

In [ ]:
# Change some other things
I.input['header']['Np'] = 10000
I.input['header']['Nx'] = 16
I.input['header']['Ny'] = 16
I.input['header']['Nz'] = 16
I.input['header']['Dt'] = 5e-13

# Other switches
#I.timeout = 1000
# Switches for MPI
I.use_mpi=True
I.input['header']['Npcol'] = 4
I.impact_bin = '$IMPACTT_MPI_BIN'
print(I)

In [ ]:
# Run
I.run()

In [ ]:
I.particles.keys()

In [ ]:
I.particles.keys()

In [ ]:
I.output['stats'].keys()

In [ ]:
I.output['slice_info']

# Plot

In [ ]:
from bokeh.plotting import output_notebook
from bokeh.plotting import show
output_notebook(hide_banner=True)

from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Range1d
from impact.parsers import load_many_fort
from impact.parsers import UNITS

from impact.plotting import layout_plot


ds = ColumnDataSource(I.output['stats'])

In [ ]:
layout = layout_plot(I.input['lattice'], plot_height=300, plot_width=600)
show(layout)

In [ ]:
def plot1(y_axis, x_axis='mean_z', source=ds):
    yunit = UNITS[y_axis]
    ylabel = y_axis+' ('+yunit+')'
    
    xunit = UNITS[x_axis]
    xlabel = x_axis+' ('+xunit+')'
    p = figure(plot_width=600, plot_height=200, x_axis_label=xlabel, y_axis_label=ylabel)
    p.line( x_axis, y_axis, source=source, color='red')
    return p
show(column(
    plot1('mean_gamma'),
    plot1('sigma_x'),
    plot1('sigma_z'),
    layout
)
)

# Archiving

In [ ]:
from impact import writers
import h5py

In [ ]:
# This is the total bunch charge in Coulombs
total_charge = I.input['header']['Bcurr'] / I.input['header']['Bfreq']
total_charge

In [ ]:
I.particles

In [ ]:
h5 = h5py.File('test.h5', 'w')

In [ ]:
I.archive(h5)

In [ ]:
h5.close()

# Reading archive

In [ ]:
# Reading input
from impact.archive import read_input_h5

In [ ]:
h5 = h5py.File('test.h5', 'r')

In [ ]:
input = read_input_h5(h5['input/'])
list(input)

In [ ]:
list(input['fieldmaps'])

# Read and Plot Particles

In [ ]:
from bokeh.plotting import  show, output_notebook
from bokeh.layouts import column, row
output_notebook(verbose=False, hide_banner=True)
import os

from pmd_beamphysics import ParticleGroup
from pmd_beamphysics.plot import marginal_plot

In [ ]:
list(h5)

In [ ]:
list(h5['output']['particles'])

In [ ]:
bunch1 = h5['output/particles/final_particles']
P =  ParticleGroup(bunch1)
marginal_plot(P, 'z', 'pz')

In [ ]:
# Cleanup
os.remove('test.h5')